In [2]:
import numpy as np
import pandas as pd
import seaborn as sns

In [3]:
import matplotlib.pyplot as plt

In [6]:
%matplotlib inline

## Step-1: run TD query and download data to local

In [11]:
query1 = '''
-- time range from 2019-08-12 to 2019-09-05
-- pull the position performance and metrics data
select pos, source_type, tid, is_pb,
       sum(impression) as total_impression, 
       sum(gmv) as total_gmv, 
       sum(add_to_cart) as total_addtocart_freq,
       sum(transaction) as total_transaction,
       sum(click_frontend) as total_click_frontend, 
       sum(click_backend) as total_click_backend
from search.daily_feed_position_metrics
where date_str >= "2019-07-01" 
      and date_str <= "2019-09-06"
--      and is_pb = 0
      and source_type IN ('tabbed_feed','search','latest')
      and invalid_session != 1
group by pos, source_type, tid, is_pb
'''

query2 = '''
-- pull EPC product data
select 
--  distinct 
--  td_time_format(td_date_trunc('month', cast(time as bigint)), 'yyyy-MM-dd') as product_month,
  product_id,
  count(product_id) as freq
from 
  sweeper.epc_product_warehouse_status 
where 
  (next_east = 'true' or next_south = 'true' or cur_south = 'true' or cur_east='true')
and
  td_time_range(time, '2019-07-01', NULL,'PDT')
group by product_id
'''

query3 = '''
-- pull data for impression logs
select td_time_format(td_date_trunc('month', cast(show_time as bigint)), 'yyyy-MM-dd') as impression_day,
       cid,
       boost_source,
       tid,
       category_id,
       feed_id,
       client,
       country_code,
       pos,
       count(request_id) as impression
from product_boost.daily_impression_truelog
where td_time_range(show_time, '2019-07-01', NULL,'PDT')
      and likely_bot = 'false'
group by 1,2,3,4,5,6,7,8,9
'''